# Coins

Chia uses a [coin set model](https://docs.chia.net/docs/04coin-set-model/intro) which means everything is a [coin](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/types/blockchain_format/coin.py) and the blockchain's state are stored in coins. 

```python
class Coin:
    parent_coin_info: bytes32
    puzzle_hash: bytes32
    amount: uint64
```

## coin id

Every Chia's coin has a unique identifier called `coin id`. The `coin id` in Chia is generated by hashing the `parent coin id`, `puzzle hash`, and `amount`. 

### Calculate coin id example

- **parent_coin_info**: `0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1`,
- **puzzle_hash**: `0x07788e204c22a929b4e28c168a7517e0d7a2c9e0bdae911fb1ca778c3968609e`
- **amount**: 42_000_000_000 (`0x09c7652400`)

The calculated **coin_id** is `0xc6d34da0b701be0e6a599ceafd61305c8df47570db58fb4e8b94527dbcdc9faf`.

#### `cdv`

In [12]:
%%bash
cdv hash 0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef107788e204c22a929b4e28c168a7517e0d7a2c9e0bdae911fb1ca778c3968609e09c7652400

c6d34da0b701be0e6a599ceafd61305c8df47570db58fb4e8b94527dbcdc9faf


#### `brun`

In [13]:
%%bash
brun '(sha256 (q . 0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1) (q . 0x07788e204c22a929b4e28c168a7517e0d7a2c9e0bdae911fb1ca778c3968609e) (q . 42_000_000_000))'

0xc6d34da0b701be0e6a599ceafd61305c8df47570db58fb4e8b94527dbcdc9faf


#### `run`

In [14]:
%%bash
run '(sha256 0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1 0x07788e204c22a929b4e28c168a7517e0d7a2c9e0bdae911fb1ca778c3968609e 42_000_000_000)'

0xc6d34da0b701be0e6a599ceafd61305c8df47570db58fb4e8b94527dbcdc9faf


#### Python

In [16]:
from chia.types.blockchain_format.sized_bytes import bytes32
from chia.util.hash import std_hash
from clvm.casts import int_to_bytes

std_hash(
    bytes32.fromhex("f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1") +
    bytes32.fromhex("07788e204c22a929b4e28c168a7517e0d7a2c9e0bdae911fb1ca778c3968609e") +
    int_to_bytes(42_000_000_000)
)

<bytes32: c6d34da0b701be0e6a599ceafd61305c8df47570db58fb4e8b94527dbcdc9faf>

#### C#
```csharp
using System.Security.Cryptography;

var parent_id = "f85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1";
var puzzle_hash = "07788e204c22a929b4e28c168a7517e0d7a2c9e0bdae911fb1ca778c3968609e";
var amount = 42_000_000_000; // in mojos

// convert values to bytes
byte[] parent_id_bytes = Convert.FromHexString(parent_id);
byte[] puzzle_hash_bytes = Convert.FromHexString(puzzle_hash);
var amount_hex_string = amount.ToString("X");
// add leading zero if needed
amount_hex_string = (amount_hex_string.Length % 2 == 0 ? "" : "0") + amount_hex_string;
byte[] amount_bytes = Convert.FromHexString(amount_hex_string);

// concat all to one bytes array
byte[] bytes = parent_id_bytes.Concat(puzzle_hash_bytes).Concat(amount_bytes).ToArray();

var hash = SHA256.Create();
byte[] coin_id_bytes = hash.ComputeHash(bytes);
string coin_id = Convert.ToHexString(coin_id_bytes);

Console.WriteLine(coin_id);
```

```sh
❯ dotnet script ./coin_id.csx
C6D34DA0B701BE0E6A599CEAFD61305C8DF47570DB58FB4E8B94527DBCDC9FAF
```

## RPC

> **Coin Record** - The format in which a coin is actually stored in the blockchain database and returned via the full node's RPC API. Contains the fundamental information about the coin as well as information about when it was confirmed and spent.

Both spent and unspent coins are stored on the blockchain and the Chia full node provides [coins RPC API](https://dkackman.github.io/chia-api/static/full_node.html#api-Coins) to retrieve [coin records](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/types/coin_record.py) in several ways including `coin id`, `puzzle hash`, `parent id` and `hint`.

In [2]:
%%bash
curl --insecure --cert ~/.chia/testnet/config/ssl/full_node/private_full_node.crt \
--key ~/.chia/testnet/config/ssl/full_node/private_full_node.key \
-H "Accept: application/json" \
-H "Content-Type: application/json" \
-X POST https://host.docker.internal:8555/get_coin_record_by_name -s \
-d '{"name": "0xc6d34da0b701be0e6a599ceafd61305c8df47570db58fb4e8b94527dbcdc9faf"}' \
| jq --color-output

{
  "coin_record": {
    "coin": {
      "amount": 42000000000,
      "parent_coin_info": "0xf85122d56db3f043af4ac7882411f7fcedf6177b1a8b9781be5f56a8668fdef1",
      "puzzle_hash": "0x07788e204c22a929b4e28c168a7517e0d7a2c9e0bdae911fb1ca778c3968609e"
    },
    "coinbase": false,
    "confirmed_block_index": 1022164,
    "spent": false,
    "spent_block_index": 0,
    "timestamp": 1653459208
  },
  "success": true
}


# References
- [Coins](https://chialisp.com/docs/coins_spends_and_wallets#coins)
- [Coin Set vs UTXO](https://docs.chia.net/docs/04coin-set-model/coin_set_vs_utxo/)
- [Coin.get_hash](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/types/blockchain_format/coin.py#L22-L30)
- [Create coin_id using c#](https://developers.chia.net/t/create-coin-id-using-c/487)